# Laboratoire 3 : Machines à vecteur de support et réseaux neuronaux
#### Département du génie logiciel et des technologies de l’information

| Étudiants             | Alexandre Bleau — BLEA14058906 / David Létinaud  — LETD05129708 / Thomas Lioret   — LIOT20069605|
|-----------------------|---------------------------------------------------------|
| Cours                 | GTI770 - Systèmes intelligents et apprentissage machine |
| Session               | Automne 2019                                            |
| Groupe                | 1                                                       |
| Numéro du laboratoire | 3                                                       |
| Professeur            | Prof. LOMBAERT                                          |
| Chargé de laboratoire | Pierre-Luc Delisle                                      |
| Date                  | 27/10/2019                                              |

## Introduction

Au cours de ce troisième laboratoire nous allons étudier deux nouveaux algorithme de classification pour résoudre le problème de classification des galaxies : les réseaux neuronaux et les machines à vecteurs de support (SVM). 
Dans un premier temps, nous allons concevoir un modèle de réseaux neuronaux basé sur le Multi-Layer Perceptron. Nous entrainerons ce modèle afin qu'il puisse classer les galaxies en "smooth" ou "spiral" en utilisant l'ensemble des primitives. Nous utiliserons le module keras de Google tensorflow
Le deuxième modèle d'apprentissage s'appuie sur un modèle d'optimisation convexe dans le cas du SVM. Dans ce cas, nous n'utiliserons qu'une partie des primitives proposées et seront couplées à nos primitives développées lors du premier laboratoire.
Nous étudierons également l'influence des hyperparamètres de ces deux méthodes afin de proposer le modèle le plus optimal dans le cas de notre problème de classification de galaxies.

## Question 1
(grille de correction : Les approches de validations sont présentées et utilisées correctement.)

Dans le cas des réseaux neuronaux, l'utilisation d'une méthode de validation croisée prendrait beaucoup de temps. En effet, il faudrait répéter plusieurs tests avec un nombre "d'epochs" conséquent : ceci serait très chronophage. Nous avons donc décider d'utiliser la méthode de validation hold-out avec 70% de donées d'entrainement et 30% de test.

Concernant la méthode SVM, nous utilisons la méthode ......... (k-fold????)

## Question 2
(La méthode de normalisation des données est décrite et correcte.)

Nous normalisons nos données grâce à la méthode "normalize" de la librairie preprocessing. La normalisation se fait par rapport à la valeur maximale. Nous avons décider de normaliser par rapport aux primitives (axis = 0). Par ailleurs, nous avons remarqué avec nos premier tests que les performances étaient meilleures si l'on normalisait par rapport à la valeur maximale.

## Question 3
(Le modèle MLP est décrit (structure, nombre de couches). La fonction de coût choisie est mentionnée ainsi que les raisons pour lesquelles elle a été choisie.)

La principale contrainte du modèle MLP est de trouver un compromis entre le temps d'apprentissage et l'accuracy. La première architecture proposée par l'énoncé est de trois couches : 100, 100, 2 (nombre de perceptrons). Voici donc nos résultats avec différentes structures de réseaux de neurones 

(RESULTATS ET COMMENTAIRES)

La fonction de coût que nous avons choisie est "Binary Cross-Entropy Loss". En effet, nous avons choisi cette fonction car nos valeurs cibles sont 0 ou 1, soit "smooth" ou "spiral".

## Question 4
(L’analyse est claire et l’équipe démontre une compréhension du phénomène de sur-apprentissage. Il le phénomène est correctement décrit et montré dans le graphique dans la mesure du possible.)

MONTRER GRAPHE DE SUR APPRENTISSAGE : GRAPHE + EXPLICATIONS

## Question 5
(1 - La démarche de comparaison des hyperparamètres est sérieuse. Les résultats sont présentés de façon correcte et concise dans un tableau et un graphique.
2 - Les explications montrant les différences sont claires, concises et plausibles.
3 - La configuration matérielle sur laquelle les expérimentations est présenteainsi que le temps d’exécution requis pour compléter les expérimentations)



## Question 6
(1 - La méthode est décrite et un lien avec l’implémentation est fait afin d’expliquer comment l’équipe a trouvé le meilleur modèle SVM. 
2- Les résultats sont présentés de façon correcte et concise dans un TABLEAU ET un GRAPHIQUEa. La configuration matérielle sur laquelle les expérimentations est présente ainsi que le temps d’exécution requis pour compléter les expérimentations.)

## Question 7
(L’impact de la taille de l’ensemble d’apprentissage sur les performances est présent et décrit convenablement.)

## Question 8
(Un classificateur est recommandé en se basant sur l’expérimentation 5 précédemment effectuée.)

## Question 9
(Des pistes d’amélioration sont proposées.)

## Conlusion

(1 - Un court résumé du problème est présent.
2 - Un rappel des résultats est présent.
3 - Des pistes pour de possibles améliorations sont présentes.)